# Импорт библиотек

In [2]:
import warnings
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, StackingClassifier, BaggingRegressor, \
    GradientBoostingRegressor, StackingRegressor
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
warnings.filterwarnings('ignore')

In [3]:
parameters_ensemble = {'n_estimators': np.arange(20,101,20),
                       'max_features': np.arange(3,24,10)}

# Датасет Adult (классификация)

In [11]:
def print_classification_model_metrics(estimator, y_test, y_pred):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(estimator.score(X_test, y_test))

In [5]:
adult_data = pd.read_csv('../data/adult_preprocessed.csv')
adult_data.head()

,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,salary,workclass_Federal-gov,workclass_Local-gov,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,39,77516,13,0,2174,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,0,13,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,1,0,0,40,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X = adult_data.drop('salary', 1).values
y = adult_data['salary'].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y)

## Базовая модель DTC

 Анализ базовой модели (обучение / поиск гиперпараметров)

In [9]:
 %%time
parameters_dtc = {'max_depth': np.arange(5,16,1)}
dtc = DecisionTreeClassifier()
dtc_base = GridSearchCV(dtc, parameters_dtc).fit(X_train, y_train)
dtc_base.best_params_

Wall time: 14 s


{'max_depth': 8}

In [12]:
 print_classification_model_metrics(dtc_base, y_test, dtc_base.predict(X_test))

[[4255  276]
 [ 633  869]]
              precision    recall  f1-score   support

           0       0.87      0.94      0.90      4531
           1       0.76      0.58      0.66      1502

    accuracy                           0.85      6033
   macro avg       0.81      0.76      0.78      6033
weighted avg       0.84      0.85      0.84      6033

0.8493286921929388


## Изучение модели BaggingClassifier

 Анализ композиции (обучение / поиск гиперпараметров)

In [13]:
 %%time
bag = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=dtc_base.best_params_['max_depth']))
model = GridSearchCV(bag, parameters_ensemble).fit(X_train, y_train)
model.best_params_

Wall time: 1min 59s


{'max_features': 23, 'n_estimators': 20}

In [14]:
print_classification_model_metrics(model, y_test, model.predict(X_test))

[[4407  124]
 [ 846  656]]
              precision    recall  f1-score   support

           0       0.84      0.97      0.90      4531
           1       0.84      0.44      0.57      1502

    accuracy                           0.84      6033
   macro avg       0.84      0.70      0.74      6033
weighted avg       0.84      0.84      0.82      6033

0.8392176363334991


## Изучение модели GradientBoostingClassifier

Анализ композиции (обучение / поиск гиперпараметров)

In [15]:
%%time
gbc = GradientBoostingClassifier()
model = GridSearchCV(gbc, parameters_ensemble).fit(X_train, y_train)
model.best_params_

Wall time: 1min 59s


{'max_features': 23, 'n_estimators': 100}

In [16]:
print_classification_model_metrics(model, y_test, model.predict(X_test))

[[4278  253]
 [ 592  910]]
              precision    recall  f1-score   support

           0       0.88      0.94      0.91      4531
           1       0.78      0.61      0.68      1502

    accuracy                           0.86      6033
   macro avg       0.83      0.78      0.80      6033
weighted avg       0.85      0.86      0.85      6033

0.8599370130946461


## Изучение модели StackingClassifier

Сделаем стекинг из двух композиций и применим DecisionTreeClassifier с найденными гиперпараметрами

In [17]:
 %%time
model = StackingClassifier(estimators=[('bag',bag), ('gbc',gbc)],
                           final_estimator=dtc_base).fit(X_train, y_train)

Wall time: 1min 21s


In [19]:
print_classification_model_metrics(model, y_test, model.predict(X_test))

[[4222  309]
 [ 537  965]]
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      4531
           1       0.76      0.64      0.70      1502

    accuracy                           0.86      6033
   macro avg       0.82      0.79      0.80      6033
weighted avg       0.85      0.86      0.86      6033

0.859771258080557


## Изучение модели CatBoostClassifier

In [20]:
 %%time
cbc = CatBoostClassifier(learning_rate=0.15).fit(X_train, y_train)
print_classification_model_metrics(cbc, y_test, cbc.predict(X_test))

0:	learn: 0.5600646	total: 162ms	remaining: 2m 41s
1:	learn: 0.4817123	total: 180ms	remaining: 1m 29s
2:	learn: 0.4302717	total: 191ms	remaining: 1m 3s
3:	learn: 0.3948494	total: 204ms	remaining: 50.7s
4:	learn: 0.3734381	total: 221ms	remaining: 44s
5:	learn: 0.3585632	total: 233ms	remaining: 38.5s
6:	learn: 0.3506159	total: 243ms	remaining: 34.4s
7:	learn: 0.3422518	total: 254ms	remaining: 31.5s
8:	learn: 0.3378749	total: 264ms	remaining: 29s
9:	learn: 0.3327641	total: 273ms	remaining: 27s
10:	learn: 0.3271629	total: 281ms	remaining: 25.3s
11:	learn: 0.3236439	total: 297ms	remaining: 24.4s
12:	learn: 0.3218725	total: 306ms	remaining: 23.2s
13:	learn: 0.3191731	total: 316ms	remaining: 22.3s
14:	learn: 0.3163681	total: 339ms	remaining: 22.2s
15:	learn: 0.3141575	total: 354ms	remaining: 21.8s
16:	learn: 0.3129260	total: 366ms	remaining: 21.2s
17:	learn: 0.3108853	total: 386ms	remaining: 21.1s
18:	learn: 0.3091555	total: 399ms	remaining: 20.6s
19:	learn: 0.3083576	total: 409ms	remaining: 

162:	learn: 0.2531038	total: 2.35s	remaining: 12.1s
163:	learn: 0.2529368	total: 2.36s	remaining: 12s
164:	learn: 0.2527225	total: 2.37s	remaining: 12s
165:	learn: 0.2525012	total: 2.38s	remaining: 11.9s
166:	learn: 0.2522921	total: 2.4s	remaining: 12s
167:	learn: 0.2519050	total: 2.41s	remaining: 11.9s
168:	learn: 0.2517300	total: 2.42s	remaining: 11.9s
169:	learn: 0.2515741	total: 2.43s	remaining: 11.9s
170:	learn: 0.2514683	total: 2.45s	remaining: 11.9s
171:	learn: 0.2513154	total: 2.46s	remaining: 11.8s
172:	learn: 0.2511482	total: 2.48s	remaining: 11.9s
173:	learn: 0.2509257	total: 2.49s	remaining: 11.8s
174:	learn: 0.2507410	total: 2.5s	remaining: 11.8s
175:	learn: 0.2506817	total: 2.51s	remaining: 11.8s
176:	learn: 0.2504581	total: 2.52s	remaining: 11.7s
177:	learn: 0.2503429	total: 2.53s	remaining: 11.7s
178:	learn: 0.2501921	total: 2.54s	remaining: 11.6s
179:	learn: 0.2499734	total: 2.57s	remaining: 11.7s
180:	learn: 0.2497735	total: 2.58s	remaining: 11.7s
181:	learn: 0.249495

326:	learn: 0.2293151	total: 4.51s	remaining: 9.27s
327:	learn: 0.2291889	total: 4.51s	remaining: 9.25s
328:	learn: 0.2291173	total: 4.53s	remaining: 9.24s
329:	learn: 0.2289412	total: 4.54s	remaining: 9.23s
330:	learn: 0.2288760	total: 4.55s	remaining: 9.21s
331:	learn: 0.2287670	total: 4.56s	remaining: 9.18s
332:	learn: 0.2286473	total: 4.58s	remaining: 9.16s
333:	learn: 0.2284890	total: 4.58s	remaining: 9.14s
334:	learn: 0.2283121	total: 4.59s	remaining: 9.12s
335:	learn: 0.2281646	total: 4.62s	remaining: 9.13s
336:	learn: 0.2280915	total: 4.63s	remaining: 9.1s
337:	learn: 0.2280040	total: 4.64s	remaining: 9.09s
338:	learn: 0.2278259	total: 4.65s	remaining: 9.07s
339:	learn: 0.2276840	total: 4.66s	remaining: 9.05s
340:	learn: 0.2275424	total: 4.67s	remaining: 9.03s
341:	learn: 0.2274106	total: 4.68s	remaining: 9s
342:	learn: 0.2272885	total: 4.69s	remaining: 8.99s
343:	learn: 0.2271825	total: 4.71s	remaining: 8.97s
344:	learn: 0.2270700	total: 4.72s	remaining: 8.96s
345:	learn: 0.22

490:	learn: 0.2114940	total: 6.65s	remaining: 6.9s
491:	learn: 0.2114322	total: 6.67s	remaining: 6.88s
492:	learn: 0.2113258	total: 6.68s	remaining: 6.87s
493:	learn: 0.2112831	total: 6.69s	remaining: 6.86s
494:	learn: 0.2111941	total: 6.7s	remaining: 6.84s
495:	learn: 0.2111095	total: 6.71s	remaining: 6.82s
496:	learn: 0.2110334	total: 6.72s	remaining: 6.8s
497:	learn: 0.2110149	total: 6.73s	remaining: 6.79s
498:	learn: 0.2109836	total: 6.74s	remaining: 6.77s
499:	learn: 0.2109176	total: 6.76s	remaining: 6.76s
500:	learn: 0.2108566	total: 6.78s	remaining: 6.75s
501:	learn: 0.2107883	total: 6.79s	remaining: 6.73s
502:	learn: 0.2106824	total: 6.8s	remaining: 6.72s
503:	learn: 0.2106152	total: 6.81s	remaining: 6.7s
504:	learn: 0.2105179	total: 6.82s	remaining: 6.68s
505:	learn: 0.2104632	total: 6.83s	remaining: 6.66s
506:	learn: 0.2103858	total: 6.83s	remaining: 6.64s
507:	learn: 0.2102961	total: 6.85s	remaining: 6.63s
508:	learn: 0.2102101	total: 6.86s	remaining: 6.62s
509:	learn: 0.210

660:	learn: 0.1960511	total: 8.81s	remaining: 4.52s
661:	learn: 0.1959820	total: 8.82s	remaining: 4.5s
662:	learn: 0.1959289	total: 8.83s	remaining: 4.49s
663:	learn: 0.1958194	total: 8.84s	remaining: 4.47s
664:	learn: 0.1956983	total: 8.86s	remaining: 4.46s
665:	learn: 0.1955646	total: 8.86s	remaining: 4.45s
666:	learn: 0.1954307	total: 8.87s	remaining: 4.43s
667:	learn: 0.1953358	total: 8.88s	remaining: 4.42s
668:	learn: 0.1953215	total: 8.89s	remaining: 4.4s
669:	learn: 0.1953040	total: 8.9s	remaining: 4.38s
670:	learn: 0.1951716	total: 8.91s	remaining: 4.37s
671:	learn: 0.1950743	total: 8.94s	remaining: 4.36s
672:	learn: 0.1949811	total: 8.95s	remaining: 4.35s
673:	learn: 0.1949485	total: 8.96s	remaining: 4.33s
674:	learn: 0.1948046	total: 8.98s	remaining: 4.32s
675:	learn: 0.1946948	total: 8.99s	remaining: 4.31s
676:	learn: 0.1945647	total: 9.01s	remaining: 4.3s
677:	learn: 0.1944419	total: 9.03s	remaining: 4.29s
678:	learn: 0.1944254	total: 9.04s	remaining: 4.28s
679:	learn: 0.19

822:	learn: 0.1837016	total: 10.9s	remaining: 2.35s
823:	learn: 0.1836157	total: 11s	remaining: 2.34s
824:	learn: 0.1835902	total: 11s	remaining: 2.33s
825:	learn: 0.1835422	total: 11s	remaining: 2.31s
826:	learn: 0.1834603	total: 11s	remaining: 2.3s
827:	learn: 0.1833476	total: 11s	remaining: 2.29s
828:	learn: 0.1831689	total: 11s	remaining: 2.27s
829:	learn: 0.1831481	total: 11s	remaining: 2.26s
830:	learn: 0.1830035	total: 11.1s	remaining: 2.25s
831:	learn: 0.1829238	total: 11.1s	remaining: 2.23s
832:	learn: 0.1828712	total: 11.1s	remaining: 2.22s
833:	learn: 0.1828065	total: 11.1s	remaining: 2.21s
834:	learn: 0.1826954	total: 11.1s	remaining: 2.19s
835:	learn: 0.1826473	total: 11.1s	remaining: 2.18s
836:	learn: 0.1826047	total: 11.1s	remaining: 2.17s
837:	learn: 0.1825038	total: 11.1s	remaining: 2.15s
838:	learn: 0.1824417	total: 11.2s	remaining: 2.14s
839:	learn: 0.1823630	total: 11.2s	remaining: 2.13s
840:	learn: 0.1822836	total: 11.2s	remaining: 2.12s
841:	learn: 0.1821676	total

995:	learn: 0.1718864	total: 13s	remaining: 52.2ms
996:	learn: 0.1718546	total: 13s	remaining: 39.2ms
997:	learn: 0.1718108	total: 13s	remaining: 26.1ms
998:	learn: 0.1717390	total: 13s	remaining: 13.1ms
999:	learn: 0.1716493	total: 13.1s	remaining: 0us
[[4205  326]
 [ 515  987]]
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      4531
           1       0.75      0.66      0.70      1502

    accuracy                           0.86      6033
   macro avg       0.82      0.79      0.81      6033
weighted avg       0.86      0.86      0.86      6033

0.8606000331510029
Wall time: 14.3 s


# Датасет Machine (регрессия)

In [21]:
 def print_regression_model_metrics(estimator, y_test, y_pred):
    print(f"Коэффициент детерминации: {estimator.score(X,y)}")
    print(f'MSE: {mean_squared_error(y_test, y_pred)}')
    print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
    print(f'MAE: {mean_absolute_error(y_test, y_pred)}')

In [22]:
machine_data = pd.read_csv('../data/machine_preprocessed.csv')
machine_data.head()

,myct,mmin,mmax,cach,chmin,chmax,prp,erp,vendor_adviser,vendor_amdahl,...,vendor_microdata,vendor_nas,vendor_ncr,vendor_nixdorf,vendor_perkin-elmer,vendor_prime,vendor_siemens,vendor_sperry,vendor_sratus,vendor_wang
0,125,256,6000,256,16,128,198,199,1,0,...,0,0,0,0,0,0,0,0,0,0
1,29,8000,32000,32,8,32,269,253,0,1,...,0,0,0,0,0,0,0,0,0,0
2,29,8000,32000,32,8,32,220,253,0,1,...,0,0,0,0,0,0,0,0,0,0
3,29,8000,32000,32,8,32,172,253,0,1,...,0,0,0,0,0,0,0,0,0,0
4,29,8000,16000,32,8,16,132,132,0,1,...,0,0,0,0,0,0,0,0,0,0


In [24]:
X = machine_data.drop('erp', 1).values
y = machine_data['erp'].values

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42)

## Базовая модель DTR

Анализ базовой модели (обучение / поиск гиперпараметров)

In [26]:
 %%time
parameters_dtr = {'max_depth': np.arange(5,16,1)}
dtr = DecisionTreeRegressor().fit(X_train, y_train)
dtr_base = GridSearchCV(dtr, parameters_dtr).fit(X_train, y_train)
print(dtr_base.best_params_)
print_regression_model_metrics(dtr_base, y_test, dtr_base.predict(X_test))

{'max_depth': 14}
Коэффициент детерминации: 0.9432995779384968
MSE: 6725.142857142857
RMSE: 82.00696834503064
MAE: 24.666666666666668
Wall time: 153 ms


## Изучение модели BaggingRegressor

Анализ композиции (обучение / поиск гиперпараметров)

In [27]:
 %%time
br = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=dtr_base.best_params_['max_depth']))
model = GridSearchCV(br, parameters_ensemble).fit(X_train, y_train)
model.best_params_

Wall time: 11.2 s


{'max_features': 23, 'n_estimators': 60}

In [28]:
 print_regression_model_metrics(model, y_test, model.predict(X_test))

Коэффициент детерминации: 0.9269299596617557
MSE: 7135.424513779546
RMSE: 84.47144200130329
MAE: 23.1098157838634


## Изучение модели GradientBoostingRegressor

Анализ композиции (обучение / поиск гиперпараметров)

In [29]:
 %%time
gbr = GradientBoostingRegressor()
model = GridSearchCV(gbr, parameters_ensemble).fit(X_train, y_train)
model.best_params_

Wall time: 2.26 s


{'max_features': 23, 'n_estimators': 80}

In [30]:
print_regression_model_metrics(model, y_test, model.predict(X_test))

Коэффициент детерминации: 0.9523898350317573
MSE: 5606.196271870593
RMSE: 74.87453687249486
MAE: 18.093193764187365


## Изучение модели StackingRegressor

Сделаем стекинг из двух композиций и применим DecisionTreeRegressor с найденными гиперпараметрами

In [31]:
%%time
model = StackingRegressor(estimators=[('br',br), ('gbr',gbr)],
                           final_estimator=dtr_base).fit(X_train, y_train)

Wall time: 736 ms


In [32]:
print_regression_model_metrics(model, y_test, model.predict(X_test))

Коэффициент детерминации: 0.8700009090769419
MSE: 7182.285714285715
RMSE: 84.74836703020132
MAE: 27.523809523809526


## Изучение модели CatBoostRegressor

In [33]:
%%time
cbr = CatBoostRegressor(learning_rate=0.15).fit(X_train, y_train)
print_regression_model_metrics(cbr, y_test, cbr.predict(X_test))

0:	learn: 115.9594972	total: 1.1ms	remaining: 1.1s
1:	learn: 106.7778338	total: 1.96ms	remaining: 976ms
2:	learn: 99.7451579	total: 2.78ms	remaining: 924ms
3:	learn: 93.7669420	total: 3.46ms	remaining: 863ms
4:	learn: 87.0848156	total: 4.11ms	remaining: 819ms
5:	learn: 81.2970261	total: 4.94ms	remaining: 818ms
6:	learn: 75.5923962	total: 5.63ms	remaining: 799ms
7:	learn: 71.2133053	total: 6.86ms	remaining: 851ms
8:	learn: 66.4562552	total: 8.38ms	remaining: 922ms
9:	learn: 63.2420088	total: 9.25ms	remaining: 915ms
10:	learn: 59.6084620	total: 10.2ms	remaining: 918ms
11:	learn: 56.3017746	total: 11.1ms	remaining: 912ms
12:	learn: 53.5098368	total: 12ms	remaining: 914ms
13:	learn: 50.3674146	total: 13.1ms	remaining: 924ms
14:	learn: 47.8663535	total: 13.8ms	remaining: 909ms
15:	learn: 45.2482275	total: 14.6ms	remaining: 896ms
16:	learn: 43.7736960	total: 15.5ms	remaining: 894ms
17:	learn: 42.1106810	total: 16.2ms	remaining: 883ms
18:	learn: 40.3202333	total: 17.4ms	remaining: 899ms
19:	l

284:	learn: 0.7376110	total: 277ms	remaining: 695ms
285:	learn: 0.7366917	total: 278ms	remaining: 694ms
286:	learn: 0.7335606	total: 279ms	remaining: 694ms
287:	learn: 0.7250282	total: 281ms	remaining: 694ms
288:	learn: 0.7210222	total: 283ms	remaining: 696ms
289:	learn: 0.7140011	total: 284ms	remaining: 695ms
290:	learn: 0.7073212	total: 285ms	remaining: 694ms
291:	learn: 0.7014565	total: 286ms	remaining: 693ms
292:	learn: 0.6994824	total: 287ms	remaining: 691ms
293:	learn: 0.6975147	total: 287ms	remaining: 690ms
294:	learn: 0.6890460	total: 289ms	remaining: 690ms
295:	learn: 0.6828987	total: 290ms	remaining: 690ms
296:	learn: 0.6761893	total: 291ms	remaining: 689ms
297:	learn: 0.6726723	total: 292ms	remaining: 688ms
298:	learn: 0.6683186	total: 293ms	remaining: 687ms
299:	learn: 0.6632188	total: 295ms	remaining: 689ms
300:	learn: 0.6566195	total: 296ms	remaining: 688ms
301:	learn: 0.6507405	total: 297ms	remaining: 686ms
302:	learn: 0.6417162	total: 298ms	remaining: 685ms
303:	learn: 

542:	learn: 0.1378763	total: 539ms	remaining: 454ms
543:	learn: 0.1366493	total: 540ms	remaining: 453ms
544:	learn: 0.1365516	total: 541ms	remaining: 451ms
545:	learn: 0.1361686	total: 542ms	remaining: 450ms
546:	learn: 0.1356169	total: 543ms	remaining: 449ms
547:	learn: 0.1349196	total: 554ms	remaining: 457ms
548:	learn: 0.1335692	total: 555ms	remaining: 456ms
549:	learn: 0.1322105	total: 556ms	remaining: 455ms
550:	learn: 0.1317541	total: 557ms	remaining: 454ms
551:	learn: 0.1315544	total: 557ms	remaining: 452ms
552:	learn: 0.1306578	total: 558ms	remaining: 451ms
553:	learn: 0.1295911	total: 560ms	remaining: 451ms
554:	learn: 0.1290244	total: 561ms	remaining: 450ms
555:	learn: 0.1284784	total: 562ms	remaining: 449ms
556:	learn: 0.1275580	total: 563ms	remaining: 448ms
557:	learn: 0.1267308	total: 563ms	remaining: 446ms
558:	learn: 0.1261707	total: 565ms	remaining: 445ms
559:	learn: 0.1257034	total: 566ms	remaining: 445ms
560:	learn: 0.1247597	total: 567ms	remaining: 444ms
561:	learn: 

801:	learn: 0.0383755	total: 824ms	remaining: 204ms
802:	learn: 0.0381326	total: 826ms	remaining: 203ms
803:	learn: 0.0379630	total: 827ms	remaining: 202ms
804:	learn: 0.0377238	total: 829ms	remaining: 201ms
805:	learn: 0.0374828	total: 829ms	remaining: 200ms
806:	learn: 0.0374441	total: 831ms	remaining: 199ms
807:	learn: 0.0374051	total: 833ms	remaining: 198ms
808:	learn: 0.0372241	total: 834ms	remaining: 197ms
809:	learn: 0.0370861	total: 835ms	remaining: 196ms
810:	learn: 0.0367148	total: 836ms	remaining: 195ms
811:	learn: 0.0365909	total: 837ms	remaining: 194ms
812:	learn: 0.0364563	total: 839ms	remaining: 193ms
813:	learn: 0.0362491	total: 841ms	remaining: 192ms
814:	learn: 0.0360948	total: 843ms	remaining: 191ms
815:	learn: 0.0360047	total: 844ms	remaining: 190ms
816:	learn: 0.0357414	total: 845ms	remaining: 189ms
817:	learn: 0.0357060	total: 846ms	remaining: 188ms
818:	learn: 0.0355555	total: 846ms	remaining: 187ms
819:	learn: 0.0354296	total: 847ms	remaining: 186ms
820:	learn: 

Коэффициент детерминации: 0.9550794265295742
MSE: 5327.953731006677
RMSE: 72.99283342223863
MAE: 16.700521986403754
Wall time: 1.66 s


# Выводы по композиционным моделям

Использование композиций алгоритмов повышает качество моделей, однако обучение происходит дольше.

Стекинг дополнительно улучшает качество моделей.

Модели catboost без подбора параметров получаются с хорошими показателями качества